<a href="https://colab.research.google.com/github/douzujun/Discourse-Relation-Code/blob/main/RoBERTa_TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# 安装必要的包

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144611 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.23-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.23-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.23-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [6]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [7]:
import os
os.chdir('/content/drive/Colab/RoBERTa/')

In [8]:
!pwd

/content/drive/Colab/RoBERTa


In [9]:
!pip install transformers

In [89]:
import torch
import pandas as pd 
import numpy as np 
import random
import matplotlib.pylab as plt 
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F 

from transformers import RobertaConfig, RobertaTokenizer, RobertaForSequenceClassification, AdamW
from transformers import RobertaModel, get_linear_schedule_with_warmup

from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

SEED = 123
BATCH_SIZE = 8

hidden_dropout_prob = 0.2
learning_rate = 5e-6
weight_decay = 1e-2
epsilon = 1e-8

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# 预处理数据

## 读取文件

In [90]:
train = pd.read_csv('./pdtb3/train.tsv', sep='\t')
dev = pd.read_csv('./pdtb3/dev.tsv', sep='\t')
test = pd.read_csv('./pdtb3/test.tsv', sep='\t') 

In [91]:
train.head()

,idx,split,section,file_number,label,category,arg1,arg2,conn,full_sense
0,0,train,2,wsj_0288,Expansion,Implicit,"The two leaders will meet on Dec. 2 and 3,",alternating the two days of meetings between a...,in particular,Expansion.Level-of-detail.Arg2-as-detail
1,1,train,2,wsj_0288,Contingency,Implicit,Despite the informal nature of the session and...,Mr. Gorbachev badly needs a diversion from the...,because,Contingency.Cause.Reason
2,2,train,2,wsj_0288,Contingency,Implicit,Mr. Gorbachev badly needs a diversion from the...,that a meeting with the leader of the U.S. cou...,thus,Contingency.Cause.Result
3,3,train,2,wsj_0288,Contingency,Implicit,"For his part, Mr. Bush has been criticized reg...",A face-to-face meeting with Mr. Gorbachev shou...,so,Contingency.Cause.Result
4,4,train,2,wsj_0288,Contingency,Implicit,"that the December meeting, will be held in the...","to hold down the ""fanfare"" and force the two s...",in order,Contingency.Purpose.Arg2-as-goal


In [92]:
dev.head()

,idx,split,section,file_number,label1,label2,category,arg1,arg2,conn1,full_sense1,conn2,full_sense2
0,0,dev,0,wsj_0068,Comparison,None,Implicit,"The Kearny, N.J.-based maker of hair accessori...","In the year-ago quarter, the company reported ...",by contrast,Comparison.Contrast,None,None
1,1,dev,0,wsj_0018,Expansion,None,Implicit,Not only is development of the new company's i...,that Cray Research Inc. will withdraw the almo...,specifically,Expansion.Level-of-detail.Arg2-as-detail,None,None
2,2,dev,0,wsj_0018,Contingency,None,Implicit,"that Seymour is the chief designer of the Cray-3,",and without him it could not be completed.,as a result,Contingency.Cause.Result,None,None
3,3,dev,0,wsj_0018,Contingency,None,Implicit,and without him it could not be completed.,Cray Research did not want to fund a project t...,so,Contingency.Cause.Result,None,None
4,4,dev,0,wsj_0018,Expansion,None,Implicit,that Cray Research's decision to link its $98....,It has to be considered as an additional risk ...,in fact,Expansion.Level-of-detail.Arg2-as-detail,None,None


In [93]:
test.head()

,idx,split,section,file_number,label1,label2,category,arg1,arg2,conn1,full_sense1,conn2,full_sense2
0,0,test,21,wsj_2131,Comparison,None,Implicit,"Among other things, the restructured facilitie...",Certain details of the restructured facilities...,although,Comparison.Concession.Arg2-as-denier,None,None
1,1,test,21,wsj_2131,Expansion,None,Implicit,Certain details of the restructured facilities...,The agreement is subject to completion of a de...,also,Expansion.Conjunction,None,None
2,2,test,21,wsj_2140,Expansion,None,Implicit,They are enjoying domestic sales that are more...,South Korean consumers are expected to buy alm...,in fact,Expansion.Level-of-detail.Arg2-as-detail,None,None
3,3,test,21,wsj_2140,Contingency,None,Implicit,that slackened demand for their cars in the U....,otherwise they wouldn't be able to keep up wit...,because,Contingency.Cause.Reason,None,None
4,4,test,21,wsj_2140,Expansion,None,Implicit,"As it is, waiting lists of a month aren't unus...",Demand is so strong that all of the domestic m...,in fact,Expansion.Level-of-detail.Arg2-as-detail,None,None


In [94]:
# ============================ 设置标签 ================================
def set_train_labels(data_set):
    data_set['multi_relation'] = None
    
    comp_idx = data_set[data_set['label'] == 'Comparison'].index    # Comparison:  0
    exp_idx = data_set[data_set['label'] == 'Expansion'].index      # Expansion:   1
    cont_idx = data_set[data_set['label'] == 'Contingency'].index   # Contingency: 2
    temp_idx = data_set[data_set['label'] == 'Temporal'].index      # Temporal:    3

    data_set.loc[:, 'multi_relation'].iloc[comp_idx] = 0
    data_set.loc[:, 'multi_relation'].iloc[exp_idx] = 1 
    data_set.loc[:, 'multi_relation'].iloc[cont_idx] = 2
    data_set.loc[:, 'multi_relation'].iloc[temp_idx] = 3 

def set_dev_or_test_labels(data_set):
    data_set['multi_relation'] = None
    
    comp_idx = data_set[data_set['label1'] == 'Comparison'].index    # Comparison:  0
    exp_idx = data_set[data_set['label1'] == 'Expansion'].index      # Expansion:   1
    cont_idx = data_set[data_set['label1'] == 'Contingency'].index   # Contingency: 2
    temp_idx = data_set[data_set['label1'] == 'Temporal'].index      # Temporal:    3

    data_set.loc[:, 'multi_relation'].iloc[comp_idx] = 0
    data_set.loc[:, 'multi_relation'].iloc[exp_idx] = 1 
    data_set.loc[:, 'multi_relation'].iloc[cont_idx] = 2
    data_set.loc[:, 'multi_relation'].iloc[temp_idx] = 3 
    
# train
set_train_labels(train)
# dev
set_dev_or_test_labels(dev)
# test
set_dev_or_test_labels(test)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [95]:
# Example: 四分类，把标签0,1,2,3 --> 换成 one-hot
F.one_hot(torch.LongTensor(test['multi_relation']), num_classes = 4)[:10, :]

tensor([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0],
        [0, 1, 0, 0]])

## BertTokenizer进行编码，将每一句转成数字

- 例子：

```python
tokenizer = RobertaTokenizer.from_pretrained(model_name, cache_dir = cache_dir)
model = RobertaForSequenceClassification.from_pretrained(model_name, return_dict=True)

inputs = tokenizer('Hello, my dog is cute', return_tensors='pt')
labels = torch.tensor([1]).unsqueeze(0)   # batch size 1, [1, 1]
outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits
```

In [96]:
model_name = 'roberta-base'
cache_dir = './Transformer-Bert-Cache/'


tokenizer = RobertaTokenizer.from_pretrained(model_name, cache_dir = cache_dir)

In [97]:
train_arg1, train_arg2, train_label = train['arg1'], train['arg2'], torch.LongTensor(train['multi_relation'])

dev_arg1, dev_arg2, dev_label = dev['arg1'], dev['arg2'], torch.LongTensor(dev['multi_relation'])

test_arg1, test_arg2, test_label = test['arg1'], test['arg2'], torch.LongTensor(test['multi_relation'])

In [98]:
train_label

tensor([1, 2, 2,  ..., 1, 0, 0])

In [99]:
# ===================== Test =========================
# t_arg1, t_arg2 = train_arg1[0], train_arg2[0]
# print(tokenizer.tokenize(t_arg1))
# print(tokenizer.tokenize(t_arg2))
# print(tokenizer.encode(t_arg1))
# print(tokenizer.encode_plus(t_arg1, t_arg2))
# print(tokenizer.convert_ids_to_tokens(tokenizer.encode(t_arg1)))
# print(tokenizer.convert_ids_to_tokens(tokenizer.encode_plus(t_arg1, t_arg2)['input_ids']))

In [100]:
# BertTokenizer进行编码，将每一句转成数字，长度不够补0
def convert_text_to_token(tokenizer, arg1, arg2, limit_size = 126):
    tokens = tokenizer.encode_plus(arg1[:limit_size], arg2[:limit_size])['input_ids']
    if len(tokens) < limit_size*2 + 4:
        tokens.extend([0] * (limit_size*2 + 4 - len(tokens)))
    return tokens

## attention_masks, 在一个文本中，如果是PAD符号则是0，否则就是1

In [101]:
def attention_masks(input_ids):
    atten_masks = []
    for seq in input_ids:
        seq_mask = [float(i > 0) for i in seq]
        atten_masks.append(seq_mask)
    return atten_masks

In [102]:
def get_inputTokens_and_attentionTokens(data_arg1, data_arg2, data_type):
    slen = len(data_arg1)
    input_ids = [convert_text_to_token(tokenizer, data_arg1[i], data_arg2[i]) for i in range(slen)]
    input_tokens = torch.tensor(input_ids)
    print(data_type, ':', input_tokens.shape)
    
    # attention_tokens
    atten_masks = attention_masks(input_ids)
    attention_tokens = torch.tensor(atten_masks)
    print(data_type, ':', attention_tokens.shape)
    
    return input_tokens, attention_tokens

In [103]:
# =================== train =====================
# train
train_input_tokens, train_attention_tokens = get_inputTokens_and_attentionTokens(train_arg1, train_arg2, train['split'][0])
# label
print('train: ', train_label.shape)

# =================== dev ===================
dev_input_tokens, dev_attention_tokens = get_inputTokens_and_attentionTokens(dev_arg1, dev_arg2, dev['split'][0])
# label
print('dev: ', dev_label.shape)

# ==================== test ==================
test_input_tokens, test_attention_tokens = get_inputTokens_and_attentionTokens(test_arg1, test_arg2, test['split'][0])
# label
print('test: ', test_label.shape)

train : torch.Size([17867, 256])
train : torch.Size([17867, 256])
train:  torch.Size([17867])
dev : torch.Size([1653, 256])
dev : torch.Size([1653, 256])
dev:  torch.Size([1653])
test : torch.Size([1474, 256])
test : torch.Size([1474, 256])
test:  torch.Size([1474])


In [104]:
# ================ Test ======================
print(tokenizer.tokenize(train_arg1[0]))
print(tokenizer.tokenize(train_arg2[0]))
print(tokenizer.encode_plus(train_arg1[0], train_arg2[0])['input_ids'])
print(train_attention_tokens[0], len(train_attention_tokens[0]))

print(train_label[0])

['The', 'Ġtwo', 'Ġleaders', 'Ġwill', 'Ġmeet', 'Ġon', 'ĠDec', '.', 'Ġ2', 'Ġand', 'Ġ3', ',']
['altern', 'ating', 'Ġthe', 'Ġtwo', 'Ġdays', 'Ġof', 'Ġmeetings', 'Ġbetween', 'Ġa', 'ĠU', '.', 'S', '.', 'Ġand', 'Ġa', 'ĠSoviet', 'Ġnaval', 'Ġvessel', 'Ġin', 'Ġthe', 'ĠMediterranean', 'ĠSea', '.']
[0, 133, 80, 917, 40, 972, 15, 1502, 4, 132, 8, 155, 6, 2, 2, 34268, 1295, 5, 80, 360, 9, 2891, 227, 10, 121, 4, 104, 4, 8, 10, 8297, 15272, 9190, 11, 5, 11965, 3939, 4, 2]
tensor([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

## 创建DataLoader，用来取出一个batch的数据

- TensorDataset 可以用来对 tensor 进行打包，就好像 python 中的 zip 功能。<br>

- 该类通过每一个 tensor 的第一个维度进行索引，所以该类中的 tensor 第一维度必须相等，且TensorDataset 中的参数必须是 tensor类型。<br>

- RandomSampler：对数据集随机采样。<br>

- SequentialSampler：按顺序对数据集采样。<br>

In [105]:
train_data = TensorDataset(train_input_tokens, train_attention_tokens, train_label)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

dev_data = TensorDataset(dev_input_tokens, dev_attention_tokens, dev_label)
dev_sampler = RandomSampler(dev_data)
dev_dataloader = DataLoader(dev_data, sampler=dev_sampler, batch_size=BATCH_SIZE)

test_data = TensorDataset(test_input_tokens, test_attention_tokens, test_label)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [106]:
# ================ Test =====================
for i, (train, mask, label) in enumerate(train_dataloader): 
    # torch.Size([16, 128]) torch.Size([16, 128]) torch.Size([16, 1])
    print(train.shape, mask.shape, label.shape)
    break
    
print('len(train_dataloader) = ', len(train_dataloader))    # 1117

torch.Size([8, 256]) torch.Size([8, 256]) torch.Size([8])
len(train_dataloader) =  2234


# 创建模型、优化器

## 创建模型

In [107]:
config = RobertaConfig.from_pretrained(model_name, num_labels = 4, 
                                       hidden_dropout_prob = hidden_dropout_prob, cache_dir=cache_dir)          

model = RobertaForSequenceClassification.from_pretrained(model_name, config=config)

model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

## 定义优化器

参数eps是为了 提高数值稳定性 而添加到分母的一个项(默认: 1e-8)。

更通用的写法：bias和LayNorm.weight没有用权重衰减

In [108]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params' : [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
     'weight_decay' : weight_decay
    },
    {'params' : [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
     'weight_decay' : 0.0
    }
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate, eps = epsilon)

## 学习率预热，训练时先从小的学习率开始训练

In [109]:
epochs = 15

# training steps 的数量: [number of batches] x [number of epochs].
total_steps = len(train_dataloader) * epochs

# 设计 learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 50, 
                                            num_training_steps = total_steps)

# 训练、评估模型

## 模型准确率, precision, recall, F1

In [110]:
def binary_acc(preds, labels):
    # print(torch.max(preds, dim=1)[1], labels)
    correct = torch.eq(torch.max(preds, dim=1)[1], labels.flatten()).float()
    acc = correct.sum().item() / len(labels)
    # print('acc:', acc)
    return acc 

# Comparison:  0
# Expansion:   1
# Contingency: 2
# Temporal:    3
def report_Evaluation(y_preds, y_labels, labels=[0, 1, 2, 3], 
                      target_names=['Comparison', 'Expansion', 'Contingency', 'Temporal'],
                      digits = 3):
    # print("eval: ", y_preds, y_labels, labels)
    return classification_report(y_preds, y_labels, labels=labels, 
                                 target_names=target_names, digits=digits)

In [111]:
# =========== Test ============
y_true = [1, 2, 0, 0, 3, 0, 3]
y_pred = [1, 2, 0, 0, 2, 0, 3]

print(report_Evaluation(y_pred, y_true))

              precision    recall  f1-score   support

  Comparison      1.000     1.000     1.000         3
   Expansion      1.000     1.000     1.000         1
 Contingency      1.000     0.500     0.667         2
    Temporal      0.500     1.000     0.667         1

    accuracy                          0.857         7
   macro avg      0.875     0.875     0.833         7
weighted avg      0.929     0.857     0.857         7



## 计算模型运行时间

In [112]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round(elapsed))
    return str(datetime.timedelta(seconds = elapsed_rounded)) # 返回 hh:mm:ss 形式的时间

## 保存和加载模型

In [113]:
def saveModel(model, name):
    torch.save(model.state_dict(), './model/' + name + '_model.pt')

def loadModel(model, name):
    model.load_state_dict(torch.load('./model/' + name + '_model.pt', map_location=device))

## 训练模型

传入model的参数必须是tensor类型的；

`nn.utils.clip_grad_norm_(parameters, max_norm, norm_type=2)` 用于解决神经网络训练过拟合的方法；

 - 输入是（NN参数，最大梯度范数，范数类型=2) 一般默认为L2 范数；
 
 - Tip： 注意这个方法只在训练的时候使用，在测试的时候不用；


In [114]:
def train_one_step(model, optimizer):
    t0 = time.time()
    avg_loss, avg_acc = [], []
    
    model.train()
    for step, batch in enumerate(train_dataloader):
        
        # 每隔40个batch 输出一下所用时间
        if step % 10 == 0 and not (step == 0):
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        
        b_input_ids, b_input_mask, b_labels = batch[0].long().to(device), batch[1].long().to(device), batch[2].long().to(device)

#         print(b_input_ids.shape, b_input_mask.shape, b_labels.shape)
        
        # loss: 损失，logits：predict
        output = model(input_ids=b_input_ids, token_type_ids=None, 
                                   attention_mask=b_input_mask, labels = b_labels)
        loss, logits = output[0], output[1]
        
        avg = binary_acc(logits, b_labels)        # (predict, label)
        avg_acc.append(avg)

        avg_loss.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 1.0)  # 大于1的梯度将其设置为1.0，以防止梯度爆炸
        optimizer.step()                          # 更新模型参数
        scheduler.step()                          # 更新learning_rate
        
        # print("loss: ", loss.item())

        # print('logits index: ', torch.max(logits, dim=1)[1])
        # print("labels: ", b_labels)
        # print('eva: ', report_Evaluation(torch.max(logits, dim=1)[1].cpu(), b_labels.cpu()))
    
    avg_acc = np.array(avg_acc).mean()
    avg_loss = np.array(avg_loss).mean()
    
    return avg_loss, avg_acc   

## 评估模型

调用model模型时不传入label值。

In [115]:
def evaluate(model, dataloader):
    avg_acc = []
    
    y_true = []
    y_pred = []
    
    model.eval()
    
    with torch.no_grad():
        for batch in dataloader:    # dev
            b_input_ids, b_input_mask, b_labels = batch[0].long().to(device), batch[1].long().to(device),                                                   batch[2].long().to(device)
            
            output = model(input_ids = b_input_ids, token_type_ids = None,
                           attention_mask = b_input_mask)
            
            logits = output[0]
            acc = binary_acc(logits, b_labels)
            avg_acc.append(acc)

            y_true.extend(b_labels.cpu().numpy())
            y_pred.extend(torch.argmax(logits, dim=1).cpu().numpy())
            
    avg_acc = np.array(avg_acc).mean()
    
    print("dev集性能: \n", report_Evaluation(y_true, y_pred))
    return avg_acc, f1_score(y_true, y_pred, average='macro')

## 测试模型

In [116]:
# test model
def test_model(model, dataloader):
    avg_acc = []
    
    y_true = []
    y_pred = []
    
    model.eval()
    
    with torch.no_grad():
        for batch in dataloader:    # dev, test
            b_input_ids, b_input_mask, b_labels = batch[0].long().to(device), batch[1].long().to(device),                                                   batch[2].long().to(device)
            
            output = model(input_ids = b_input_ids, token_type_ids = None,
                           attention_mask = b_input_mask)
            
            logits = output[0]
            acc = binary_acc(logits, b_labels)
            avg_acc.append(acc)

            y_true.extend(b_labels.cpu().numpy())
            y_pred.extend(torch.argmax(logits, dim=1).cpu().numpy())
            
    avg_acc = np.array(avg_acc).mean()
    
    print("Test集性能: \n", report_Evaluation(y_true, y_pred))
    return avg_acc, f1_score(y_true, y_pred, average='macro')

# 训练模型和评估模型

In [117]:
def Train():
    best_dev_f1 = float("-inf")

    for epoch in range(epochs):
        train_loss, train_acc = train_one_step(model, optimizer)
        print('epoch={}, 训练准确率={}，损失={}'.format(epoch, train_acc, train_loss))
        
        dev_acc, dev_f1 = evaluate(model, dev_dataloader)
        print("epoch={}, 开发集准确率={}, F1值={}".format(epoch, dev_acc, dev_f1))

        if dev_f1 > best_dev_f1:
            best_dev_f1 = dev_f1
            saveModel(model, 'roberta')

In [118]:
def Test():
    loadModel(model, 'roberta')
    test_acc, test_F1 = test_model(model, test_dataloader)
    print("Test. Acc: {0:.2f}%.\nTest. macro f1-score: {1:.2f}.".format(test_acc * 100, test_F1))

In [ ]:
Train()

Test()

  Batch    10  of  2,234.    Elapsed: 0:00:04.
  Batch    20  of  2,234.    Elapsed: 0:00:08.
  Batch    30  of  2,234.    Elapsed: 0:00:12.
  Batch    40  of  2,234.    Elapsed: 0:00:16.
  Batch    50  of  2,234.    Elapsed: 0:00:20.
  Batch    60  of  2,234.    Elapsed: 0:00:23.
  Batch    70  of  2,234.    Elapsed: 0:00:27.
  Batch    80  of  2,234.    Elapsed: 0:00:31.
  Batch    90  of  2,234.    Elapsed: 0:00:35.
  Batch   100  of  2,234.    Elapsed: 0:00:39.
  Batch   110  of  2,234.    Elapsed: 0:00:43.
  Batch   120  of  2,234.    Elapsed: 0:00:48.
  Batch   130  of  2,234.    Elapsed: 0:00:52.
  Batch   140  of  2,234.    Elapsed: 0:00:56.
  Batch   150  of  2,234.    Elapsed: 0:01:00.
  Batch   160  of  2,234.    Elapsed: 0:01:04.
  Batch   170  of  2,234.    Elapsed: 0:01:08.
  Batch   180  of  2,234.    Elapsed: 0:01:12.
  Batch   190  of  2,234.    Elapsed: 0:01:16.
  Batch   200  of  2,234.    Elapsed: 0:01:20.
  Batch   210  of  2,234.    Elapsed: 0:01:24.
  Batch   220